In [1]:
import ccxt


exchange = ccxt.binance()

symbols = exchange.load_markets()

symbols_to_trade = list(filter(lambda coin: '/USDT' in coin, exchange.symbols))
symbols_to_trade

['1INCH/USDT',
 '1INCHDOWN/USDT',
 '1INCHUP/USDT',
 'AAVE/USDT',
 'AAVEDOWN/USDT',
 'AAVEUP/USDT',
 'ACM/USDT',
 'ADA/USDT',
 'ADADOWN/USDT',
 'ADAUP/USDT',
 'AION/USDT',
 'AKRO/USDT',
 'ALGO/USDT',
 'ALICE/USDT',
 'ALPACA/USDT',
 'ALPHA/USDT',
 'ANKR/USDT',
 'ANT/USDT',
 'AR/USDT',
 'ARDR/USDT',
 'ARPA/USDT',
 'ASR/USDT',
 'ATA/USDT',
 'ATM/USDT',
 'ATOM/USDT',
 'AUD/USDT',
 'AUDIO/USDT',
 'AUTO/USDT',
 'AVA/USDT',
 'AVAX/USDT',
 'AXS/USDT',
 'BADGER/USDT',
 'BAKE/USDT',
 'BAL/USDT',
 'BAND/USDT',
 'BAR/USDT',
 'BAT/USDT',
 'BCC/USDT',
 'BCH/USDT',
 'BCHDOWN/USDT',
 'BCHUP/USDT',
 'BEAM/USDT',
 'BEAR/USDT',
 'BEL/USDT',
 'BKRW/USDT',
 'BLZ/USDT',
 'BNB/USDT',
 'BNBBEAR/USDT',
 'BNBBULL/USDT',
 'BNBDOWN/USDT',
 'BNBUP/USDT',
 'BNT/USDT',
 'BOND/USDT',
 'BSV/USDT',
 'BTC/USDT',
 'BTCDOWN/USDT',
 'BTCST/USDT',
 'BTCUP/USDT',
 'BTG/USDT',
 'BTS/USDT',
 'BTT/USDT',
 'BULL/USDT',
 'BURGER/USDT',
 'BUSD/USDT',
 'BZRX/USDT',
 'C98/USDT',
 'CAKE/USDT',
 'CELO/USDT',
 'CELR/USDT',
 'CFX/USDT',


In [2]:
def fetch_data(symbol):
    
    
    import time
    import pandas as pd
    from datetime import datetime
    from IPython.display import clear_output
    
    start = time.time()


    from_datetime = '2020-01-01 00:00:00'
    from_timestamp = exchange.parse8601(from_datetime)

    msec = 1000
    minute = 60 * msec

    now = exchange.milliseconds()
    
    df = pd.DataFrame(columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df.set_index('Time', inplace=True)
    
    while from_timestamp < now:
    
        print('Fetching candles starting from', exchange.iso8601(from_timestamp), f'for symbol {symbol}')
        data = exchange.fetch_ohlcv(symbol, '1m', from_timestamp, limit=1000)
        if len(data):
            
            data = pd.DataFrame(data, columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
            data.Time = data.Time.astype('Int64')
            data['Time'] = data.apply(lambda row: datetime.fromtimestamp(row.Time / 1e3), axis=1)
            data.set_index('Time', inplace=True)
            df = df.append(data)
        
        else:
            from_timestamp += 1000 * minute
            
        time.sleep(2*exchange.rateLimit / msec)
        from_timestamp += len(data) * minute
        #print(df)
        clear_output()
    symbol = symbol.replace('/', '_')
    df.to_csv(f'crypto_data/{symbol}.csv')
    #pbar.update(1)
    print(f'Time taken: {time.time() - start}')
    return 'DONE'
    
    


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from time import time
from tqdm import tqdm





start = time()

processes = []
with ThreadPoolExecutor(max_workers=10) as executor:
    for symbol in tqdm(symbols_to_trade):
        processes.append(executor.submit(fetch_data, symbol))

for task in as_completed(processes):
    print(task.result())


print(f'Time taken: {time() - start}')

100%|██████████| 334/334 [00:00<00:00, 7554.00it/s]
